In [1]:
import pandas as pd
import os
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
import gensim
import numpy as np
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary

In [2]:
!wget --no-check-certificate \
 https://storage.googleapis.com/txt_tesis/txts_extracted.zip \
 -O /tmp/files.zip

--2020-09-22 12:20:25--  https://storage.googleapis.com/txt_tesis/txts_extracted.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2887953 (2.8M) [application/zip]
Saving to: ‘/tmp/files.zip’

/tmp/files.zip      100%[===================>]   2.75M  --.-KB/s    in 0.02s   

2020-09-22 12:20:25 (173 MB/s) - ‘/tmp/files.zip’ saved [2887953/2887953]



In [3]:
!unzip /tmp/files.zip -d /tmp/

Archive:  /tmp/files.zip
   creating: /tmp/txt_tesis/
  inflating: /tmp/txt_tesis/acta_comision_2020_01_21_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_09_04_a_comi_credenciales.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_07_15_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_07_23_a_comi_credenciales.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_09_18_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_10_29_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_10_24_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_comision_2020_01_22_a_comi_canal.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_09_05_a_comi_credenciales.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_07_18_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_07_30_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_09_03_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_12_04_a_pleno.txt  
  inflating: /tmp

In [19]:
dataPath='/tmp/txt_tesis/'
data=''

In [23]:
docs=[]
txtNames=[]
for txtName in os.listdir(dataPath):
    if('comision' in txtName):
      txtNames.append(txtName)

for txtName in txtNames:
    txtFile=open(dataPath+txtName)
    txtFile.seek(0)
    docs.append(txtFile.read())
    txtFile.close()

In [26]:
comisiones={}
p = re.compile('\w+comi_(\w+)')
for txt in txtNames:
  if('comision' in txt):
    result = p.search(txt)
    comision = result.group(1)
    if comision not in comisiones:
      comisiones[comision]=1
    else:
      value= comisiones.get(comision)
      comisiones[comision]=value+1  

In [28]:
len(comisiones)

13

In [29]:
comisiones

{'canal': 8,
 'comercio': 5,
 'comunicacion': 8,
 'credenciales': 23,
 'educacion': 7,
 'familia': 5,
 'indigena': 1,
 'justicia': 16,
 'municipales': 4,
 'poblacion': 5,
 'presupuesto': 1,
 'relaciones': 5,
 'trabajo': 4}

In [30]:
black_list=['diputado','secretario','punto','día','dia','acta','sesion','honorable','señor','periodo','sesiones','presidente','presidenta','sesion','sesiones','correspondencia','orden','pleno','proyecto','ministro','diputados','miembros','miembro','proyectos','funcionario','funcionarios','anteproyecto','anteproyectos']

In [31]:
def cleaner(word):
  word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', word, flags=re.MULTILINE)#Direcciones Web. 
  word = re.sub(r'\#\.', '', word)#Numerales seguidos de puntos.
  word = re.sub(r'\n', '', word)#Saltos de linea
  word = re.sub(r',','', word)#Comas
  word = re.sub(r'\-', ' ', word)#Guiones.
  word = re.sub(r'\.{3}', ' ', word)#Puntos suspensivos.
  word = re.sub(r'á', 'a', word)#tildes.
  word = re.sub(r'é', 'e', word)
  word = re.sub(r'í', 'i', word)
  word = re.sub(r'ó', 'o', word)
  word = re.sub(r'ú', 'u', word) 
  word = re.sub(r'a{2,}', 'a', word)#Dos o más a seguidas -->a 
  word = re.sub(r'e{2,}', 'e', word)#Dos o más e seguidas -->a 
  word = re.sub(r'i{2,}', 'i', word)#Dos o más i seguidas -->a 
  word = re.sub(r'o{2,}', 'o', word)#Dos o más o seguidas -->a 
  word = re.sub(r'u{2,}', 'u', word)#Dos o más u seguidas -->a 
  word = re.sub(r'ja{2,}', 'ja', word)#Jajaja -->ja  
  word = re.sub('[^a-zA-Z]', ' ', word)#Todo lo que no sea palabras. 
  list_word_clean = []
  for w1 in word.split(" "):
    if  w1.lower() not in stopwords:
      list_word_clean.append(w1.lower())

  bigram_list = bigram[list_word_clean]
  out_text = lemmatization(" ".join(bigram_list))
  return out_text

In [32]:
def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = [ token.text for token in nlp(texts) if token.pos_ in 
                 allowed_postags and token.text not in black_list and len(token.text)>2]
    return texts_out

In [33]:
bigram = gensim.models.Phrases(docs)

In [34]:
import spacy.cli
spacy.cli.download("es_core_news_md")
import spacy
nlp = spacy.load('es_core_news_md')
import nltk
nltk.download('stopwords')

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('spanish'))

additional_stopwords=set(black_list)

stopwords = stop.union(additional_stopwords)


In [36]:
docsCleaned=[]
for doc in docs:
    docsCleaned.append(cleaner(doc))

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [37]:
dictionary = Dictionary(docsCleaned)

In [42]:

dictionary.compactify()#Da nuevos ID. 
#Filtramos palabras que estén en minimo 2 documentos y máximo el 97% de documentos. 
dictionary.filter_extremes(no_below=2, no_above=0.90, keep_n=None)
dictionary.compactify()
#Creamos el Corpus
corpus = [dictionary.doc2bow(text) for text in docsCleaned]

In [43]:
#Función para mostrar los temas. 
def display_topics(model, model_type="lda"):
  for topic_idx, topic in enumerate(model.print_topics()):
    print ("Topic %d:" % (topic_idx))
    if model_type== "hdp":
      print (" ".join(re.findall( r'\*(.[^\*-S]+).?', topic[1])), "\n")
    else:
      print (" ".join(re.findall( r'\"(.[^"]+).?', topic[1])), "\n")

In [44]:
hdpmodel=HdpModel(corpus=corpus,id2word=dictionary,random_state=30)

In [45]:
display_topics(hdpmodel,model_type="hdp")

Topic 0:
personas  transporte  debate  palabra  carga  gente  cosas  desarrollo  forma  persona 

Topic 1:
palabra  programa  tiempo  personas  oportunidad  acuerdo  trabajo  animales  apoyo  vida 

Topic 2:
licencia  tipo  agua  transporte  problema  director  vez  cosas  palabra  personas 

Topic 3:
carga  transporte  desarrollo  peso  dimensiones  vehiculo  agua  personas  debate  leyes 

Topic 4:
cultura  articulo  contrataciones  palabra  personas  propuestas  acto  empresas  propuesta  autor 

Topic 5:
personas  debate  persona  palabra  trabajo  titulo  caso  discapacidad  tipo  noviembre 

Topic 6:
zona  sector  productor  propuesta  palabra  personas  productores  recursos  banco  gerente 

Topic 7:
palabra  uso  debate  agua  personas  tiempo  recursos  momento  diputada  municipios 

Topic 8:
papel  debate  doctor  justicia  reformas  temas  uso  palabra  vez  documento 

Topic 9:
dispositivos  productos  salud  nivel  problema  basura  registro  artistas  diciembre  propues

In [47]:
lsimodel = LsiModel(corpus=corpus, num_topics=13, id2word=dictionary)

In [48]:
display_topics(lsimodel)

Topic 0:
personas palabra tipo debate tiempo momento persona vez cosas licencia 

Topic 1:
licencia tipo transporte vehiculo palabra examenes autoridad carga puntos pasajeros 

Topic 2:
carga cultura transporte licencia tipo articulo dimensiones personas peso edad 

Topic 3:
cultura carga articulo transporte agua personas contrataciones licencia dimensiones productos 

Topic 4:
cultura persona personas delitos agua armas articulo edad delito defensa 

Topic 5:
contrataciones productos agua cultura reglamento propuestas propuesta precio dispositivos salud 

Topic 6:
contrataciones salud productos dispositivos reglamento palabra problema acto personas cargos 

Topic 7:
agua uso debate director trabajo medicamentos salud registro ministra personas 

Topic 8:
personas armas defensa debate reglamento tiempo agua juez contrataciones palabra 

Topic 9:
productor sector banco agua productores banca compras alcance propuesta contrataciones 

Topic 10:
delitos armas reglamento edad defensa abuso

In [50]:
ldamodel= LdaModel(corpus=corpus,num_topics=13,id2word=dictionary)

In [51]:
display_topics(ldamodel)

Topic 0:
personas debate palabra tiempo persona cosas transporte acuerdo vida nivel 

Topic 1:
debate personas tipo palabra tiempo persona vez forma trabajo acuerdo 

Topic 2:
personas palabra debate tipo tiempo vida cosas momento vez persona 

Topic 3:
personas palabra momento tiempo nivel vez uso requisitos agua gente 

Topic 4:
personas palabra transporte persona vez debate tiempo momento sector acuerdo 

Topic 5:
palabra personas momento nivel vez vida agua tiempo forma debate 

Topic 6:
personas debate palabra tiempo momento nivel vez reglamento vida uso 

Topic 7:
personas palabra tiempo uso debate vida momento desarrollo vez caso 

Topic 8:
personas debate momento palabra uso persona trabajo tipo reglamento tiempo 

Topic 9:
palabra personas vez momento debate persona tiempo cosas ejemplo tipo 

Topic 10:
personas palabra reglamento tipo licencia persona vez cosas tiempo momento 

Topic 11:
palabra personas debate tiempo momento vez cosas caso secretaria uso 

Topic 12:
personas